# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [1]:
# import libraries
import numpy as np
import pandas as pd
import sys
from sqlalchemy import create_engine
import nltk
import joblib

nltk.download('punkt')
nltk.download('stopwords')

from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem.porter import PorterStemmer

import re

from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer, TfidfTransformer
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier, AdaBoostClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import LogisticRegression
from sklearn.decomposition import TruncatedSVD

from imblearn.over_sampling import SMOTE
from imblearn.pipeline import Pipeline as iPipeline

from sklearn.multioutput import MultiOutputClassifier
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report, precision_score, recall_score, f1_score

from mismote import MLSMOTE

[nltk_data] Downloading package punkt to /home/salman1567/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /home/salman1567/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


## Part I: Loading the Dataset

In [2]:
# load data from database
engine = create_engine('sqlite:///disaster.db')
df = pd.read_sql_table('disaster', engine)
X = df.iloc[:,1]
Y = df.iloc[:,4:]

# We can drop child alone since there aren't any messages with that category.


print(X.head())

0    Weather update - a cold front from Cuba that c...
1              Is the Hurricane over or is it not over
2                      Looking for someone but no name
3    UN reports Leogane 80-90 destroyed. Only Hospi...
4    says: west side of Haiti, rest of the country ...
Name: message, dtype: object


This dataset is imbalanced so we will try to attempt to merge some of the really small categories which have less than 200 samples. This does mean we lose some specificity with our model and this is an important consideration if this trade off is acceptable or not. It can be argued in some situations we don't need to know the exact specific disaster but rather the general category such as if it is infrastructural related we make sure we send out a team with tools and general capability to handle that kind of situation.

We will be merging buildings, electricity, tools, hospitals, shops, aid-centers, other_infrastructure all into `infrastructure_related`. The reason for doing this makes sense from a logical point of view since those are all infrastructure related and will typically mean we send out similar teams prepared for any kind of infrastructure related problems.

We will also be placing clothing, money, missing people, refugees and death category into `other_aid`



## Part II: Experimenting with Feature Engineering

In [3]:
# If any of these features are 1 than infrastructure_related should be 1 since we merging them in


df.loc[(df['buildings']==1) |
      (df['electricity']==1) |
      (df['tools']==1) |
      (df['hospitals']==1) |
      (df['shops']==1) |
      (df['aid_centers']==1) |
      (df['other_infrastructure']==1), 'infrastructure_related'] = 1

# Merging categories into other aid

df.loc[(df['clothing']==1) |
      (df['money']==1) |
      (df['missing_people']==1) |
      (df['refugees']==1) |
      (df['death']==1), 'other_aid'] = 1

# Merging floods, storm, cold and other_weather into weather_related

df.loc[(df['floods']==1) |
      (df['storm']==1) |
      (df['cold']==1) |
      (df['other_weather']==1), 'weather_related'] = 1

# for i in df.columns.unique()[7:]:
#       print(df.columns.unique())
#       if df[df[i]==1].shape[0] > 1000:
#             pass
#       else:
#             df = df.drop(labels=[i])


# Dropping the merged in features
df = df.drop(labels=['buildings', 'electricity', 'tools', 'hospitals', 'shops', 'aid_centers', 'other_infrastructure',
      'clothing', 'money', 'missing_people', 'refugees', 'death', 'floods', 'storm', 'cold', 'other_weather'], axis=1)



#### Feature merging

Later when running the pipeline we see that this merging improves f1 score very slightly, lowering precision but improving recall thus overall with an improved f1 score. Since we are dealing with an imbalanced dataset we will prioritize recall and f1 score over precision.

In [4]:
# Making sure that the features successfuly merged into infrastructure_related
X = df.iloc[:,1]
Y = df.iloc[:,4:]

for i in Y.columns.unique():
    print(i, Y[Y[i]==1].shape[0], sep=" - ")

related - 19914
request - 4480
offer - 119
aid_related - 10878
medical_help - 2087
medical_products - 1314
search_and_rescue - 724
security - 471
military - 860
water - 1674
food - 2930
shelter - 2319
other_aid - 5737
infrastructure_related - 3064
transport - 1203
weather_related - 7302
fire - 282
earthquake - 2453
direct_report - 5080


In [5]:
# Do the same for other aid
Y.head()

,related,request,offer,aid_related,medical_help,medical_products,search_and_rescue,security,military,water,food,shelter,other_aid,infrastructure_related,transport,weather_related,fire,earthquake,direct_report
0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,1,0,0,1,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0
2,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,1,1,0,1,0,1,0,0,0,0,0,0,1,1,0,0,0,0,0
4,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


## Part III: Tokenization of text data
### 2. Write a tokenization function to process your text data

In [6]:
def tokenize(text):
    
    clean_tokens = []
    
    # lowercasing and removing punctuation
    text = re.sub(r"[^a-zA-Z0-9]", " ", text.lower().strip())
    tokens = word_tokenize(text)
    # Remove stop words + stem
    stemmer = PorterStemmer()
    
    
    stop_words = stopwords.words("english")
    clean_tokens = [ stemmer.stem(word) for word in tokens if word not in stop_words]

    return clean_tokens


### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [7]:
# build a transformer that scales
class OverSamplingTransformer(BaseEstimator, TransformerMixin):
    """A custom transformer to be used in the ML pipeline to handle the class imbalance that is present
    in this dataset.
    """
    
    def __init__(self, n_sample):
        self.n_sample = n_sample
        pass
    
    def fit(self, X, y=None):
        X = pd.DataFrame(X)
        y = pd.DataFrame(y)
        return MLSMOTE(X, y, self.n_sample)
    
    def transform(self, X):
        pass

In [8]:
# Try other estimators to see if performance is better
# Implement oversampling using SVD of n components

# Number of featurs we want to reduce to so that we can then use this in oversampling transformer
n_features = 33 

pipeline = iPipeline([
    #('smt', SMOTE()),
    ('features', FeatureUnion([
        ('nltk_pipeline', Pipeline([
                ('count-vector', CountVectorizer(tokenizer=tokenize)),
                ('tf-idf', TfidfTransformer())   
        ])),
        
        
        
    ])),
     #('tf-idf', TfidfVectorizer()),
    ('SVD', TruncatedSVD(n_components=n_features)),
    #('smote', OverSamplingTransformer(n_sample=1000)),
    ('clf', MultiOutputClassifier(RandomForestClassifier(), n_jobs=-1))
])

In [9]:
# Trying a different model
pipeline2 = Pipeline([
    #('smt', SMOTE()),
    ('count-vector', CountVectorizer(tokenizer=tokenize)),
    ('tf-idf', TfidfTransformer()),
    #('tf-idf', TfidfVectorizer()),
    ('clf', MultiOutputClassifier(AdaBoostClassifier(), n_jobs=-1))
])

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [10]:
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.33, random_state=1)
pipeline.fit(X_train, y_train)


Pipeline(steps=[('features',
                 FeatureUnion(transformer_list=[('nltk_pipeline',
                                                 Pipeline(steps=[('count-vector',
                                                                  CountVectorizer(tokenizer=<function tokenize at 0x7f4e248d1e10>)),
                                                                 ('tf-idf',
                                                                  TfidfTransformer())]))])),
                ('SVD', TruncatedSVD(n_components=33)),
                ('clf',
                 MultiOutputClassifier(estimator=RandomForestClassifier(),
                                       n_jobs=-1))])

In [11]:
print(X_train)

9979     7.0 earthquake rocks Haiti The Christian Scien...
3632     I WOULD BE HAPPY BECAUSE EVEN COROSSOL JUICE,T...
18789    Because of this, at 9:05am a safety valve on t...
21277    The state power company Tenaga Nasional Bhd ha...
15451    USAID, through the Assistance for Emergency Lo...
                               ...                        
10955    Really hope this Hurricane doesn't happen . #f...
17289    In Xai-Xai, a small city about 90 miles from M...
5192     I didn't ever get an answer to the question I ...
12172    shamsher haider mirani moza sumra nashaib janoobi
235      Send cards ( airtime, calling cards ) for Digi...
Name: message, Length: 17430, dtype: object


### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [12]:
col_names = Y.columns.unique()
y_pred = pipeline.predict(X_test)
print(classification_report(y_test, y_pred, target_names=col_names))
    



                        precision    recall  f1-score   support

               related       0.77      0.97      0.86      6545
               request       0.46      0.08      0.14      1520
                 offer       0.00      0.00      0.00        34
           aid_related       0.46      0.23      0.30      3582
          medical_help       0.00      0.00      0.00       672
      medical_products       0.33      0.00      0.01       441
     search_and_rescue       0.50      0.00      0.01       230
              security       0.00      0.00      0.00       155
              military       1.00      0.00      0.01       308
                 water       0.00      0.00      0.00       568
                  food       0.27      0.01      0.01       973
               shelter       0.08      0.00      0.00       775
             other_aid       0.24      0.01      0.02      1877
infrastructure_related       0.19      0.00      0.01      1013
             transport       0.00      

/home/salman1567/code/Data Science - Machine Learning Projects/Data Science Portfolio/Diasaster Response Classification/env/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/salman1567/code/Data Science - Machine Learning Projects/Data Science Portfolio/Diasaster Response Classification/env/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/salman1567/code/Data Science - Machine Learning Projects/Data Science Portfolio/Diasaster Response Classification/env/lib/python3.10/site-packag

In [13]:
X_train2, X_test2, y_train2, y_test2 = train_test_split(X, Y, test_size=0.33, random_state=1)
pipeline2.fit(X_train2, y_train2)
col_names = Y.columns.unique()
y_pred2 = pipeline2.predict(X_test2)
print(classification_report(y_test2, y_pred2, target_names=col_names))

ValueError: WRITEBACKIFCOPY base is read-only

In [ ]:
# Checking what hyper parameters can be tuned
pipeline.get_params().keys()

dict_keys(['memory', 'steps', 'verbose', 'count-vector', 'tf-idf', 'clf', 'count-vector__analyzer', 'count-vector__binary', 'count-vector__decode_error', 'count-vector__dtype', 'count-vector__encoding', 'count-vector__input', 'count-vector__lowercase', 'count-vector__max_df', 'count-vector__max_features', 'count-vector__min_df', 'count-vector__ngram_range', 'count-vector__preprocessor', 'count-vector__stop_words', 'count-vector__strip_accents', 'count-vector__token_pattern', 'count-vector__tokenizer', 'count-vector__vocabulary', 'tf-idf__norm', 'tf-idf__smooth_idf', 'tf-idf__sublinear_tf', 'tf-idf__use_idf', 'clf__estimator__bootstrap', 'clf__estimator__ccp_alpha', 'clf__estimator__class_weight', 'clf__estimator__criterion', 'clf__estimator__max_depth', 'clf__estimator__max_features', 'clf__estimator__max_leaf_nodes', 'clf__estimator__max_samples', 'clf__estimator__min_impurity_decrease', 'clf__estimator__min_samples_leaf', 'clf__estimator__min_samples_split', 'clf__estimator__min_weig

`TODO`: Consider to remove child_alone since there are none of those and also consider oversampling since some of the disaster categories have a low amount of samples.

In [ ]:
# we can consider dropping any columns that have zero rows associated
for i in df.columns.unique():
    if sum(df[i]) < 200:
        df = df.drop(columns=[i])
        


### 6. Improve your model
Use grid search to find better parameters. 

In [ ]:
parameters = {#'clf__estimator__criterion': ("squared_error", 'absolute_error', 'poisson'),
              #'clf__estimator__max_features': ('auto', 'sqrt', 'log2'), 
              'clf__n_jobs': [-1], 
              'clf__estimator__n_jobs': [-1]}

cv = GridSearchCV(pipeline, parameters)
cv.fit(X_train, y_train)

GridSearchCV(estimator=Pipeline(steps=[('count-vector',
                                        CountVectorizer(tokenizer=<function tokenize at 0x7f99541a2710>)),
                                       ('tf-idf', TfidfTransformer()),
                                       ('clf',
                                        MultiOutputClassifier(estimator=RandomForestClassifier(),
                                                              n_jobs=-1))]),
             param_grid={'clf__estimator__n_jobs': [-1], 'clf__n_jobs': [-1]})

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [ ]:
y_pred = cv.predict(X_test)
print(classification_report(y_test, y_pred, target_names=col_names))

                        precision    recall  f1-score   support

               related       0.77      0.97      0.86      6545
               request       0.46      0.05      0.09      1520
                 offer       0.00      0.00      0.00        34
           aid_related       0.46      0.21      0.29      3582
          medical_help       0.08      0.00      0.01       672
      medical_products       0.00      0.00      0.00       441
     search_and_rescue       0.17      0.00      0.01       230
              security       0.00      0.00      0.00       155
              military       1.00      0.00      0.01       308
                 water       0.04      0.00      0.00       568
                  food       0.28      0.01      0.03       973
               shelter       0.10      0.00      0.01       775
              clothing       0.00      0.00      0.00       134
                 money       0.00      0.00      0.00       188
        missing_people       0.00      

/home/salman1567/code/Data Science - Machine Learning Projects/Data Science Portfolio/Diasaster Response Classification/env/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/salman1567/code/Data Science - Machine Learning Projects/Data Science Portfolio/Diasaster Response Classification/env/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/salman1567/code/Data Science - Machine Learning Projects/Data Science Portfolio/Diasaster Response Classification/env/lib/python3.10/site-packag

We notice that a lot of the scores have 0 since we have a imbalanced data set. We can try to fix this by oversampling

### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

In [ ]:
# Check what params gridsearchcv has selected
cv.get_params()

{'cv': None,
 'error_score': nan,
 'estimator__memory': None,
 'estimator__steps': [('count-vector',
   CountVectorizer(tokenizer=<function tokenize at 0x7f99541a2710>)),
  ('tf-idf', TfidfTransformer()),
  ('clf',
   MultiOutputClassifier(estimator=RandomForestClassifier(), n_jobs=-1))],
 'estimator__verbose': False,
 'estimator__count-vector': CountVectorizer(tokenizer=<function tokenize at 0x7f99541a2710>),
 'estimator__tf-idf': TfidfTransformer(),
 'estimator__clf': MultiOutputClassifier(estimator=RandomForestClassifier(), n_jobs=-1),
 'estimator__count-vector__analyzer': 'word',
 'estimator__count-vector__binary': False,
 'estimator__count-vector__decode_error': 'strict',
 'estimator__count-vector__dtype': numpy.int64,
 'estimator__count-vector__encoding': 'utf-8',
 'estimator__count-vector__input': 'content',
 'estimator__count-vector__lowercase': True,
 'estimator__count-vector__max_df': 1.0,
 'estimator__count-vector__max_features': None,
 'estimator__count-vector__min_df': 1,


### 9. Export your model as a pickle file

In [ ]:
# Export the model as a pickle file
joblib.dump(cv, 'classifier.pkl')

['classifier.pkl']

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.

In [ ]:
model = joblib.load('classifier.pkl')

In [ ]:
model.predict(["Weather update - a cold front from Cuba that could pass over Haiti"])


array([[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0]])

In [ ]:
y_test

,related,request,offer,aid_related,medical_help,medical_products,search_and_rescue,security,military,water,...,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
20397,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
16452,1,0,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,1,0,1,0
13143,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
22201,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
25308,1,0,0,1,0,0,0,0,0,0,...,0,1,1,0,0,0,1,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6910,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
21432,1,0,0,1,0,0,0,0,0,1,...,0,0,1,1,0,0,0,1,0,0
17497,1,0,0,1,0,0,0,0,0,0,...,0,0,1,0,0,0,1,0,0,0
22275,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [ ]:
X_test[0]

'Weather update - a cold front from Cuba that could pass over Haiti'